In [1]:
# 09_06.md 이어서 작성하는 실습 코드

## 5. GloVe 훈련시키기
---

In [5]:
# 글로브 패키지 설치
# !pip install glove_python > 로 설치 안됨. 2016년도에 마지막으로 릴리즈됨
# >> pip install glove_python_binary 으로 대신 설치하면 심신의 평화를 찾을 수 있음
# !pip install glove_python_binary

  Using cached glove_python_binary-0.2.0-cp38-cp38-win_amd64.whl (244 kB)


### 09_04 에서 썼던 데이터 준비
---

In [9]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [10]:
datapath = 'D:/chch/startwhithtorch/datafolder/'

# 훈련 데이터 전처리하기
targetXML = open(datapath + 'ted_en-2016408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

parse_text = '/n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'/([^)]*/)', '', parse_text)
# parse_text 를 불러와서 (Audio), (Laughter) 등의 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거한다는 뜻

sent_text = sent_tokenize(content_text)

normalized_text = []
for string in sent_text :
    tokens = re.sub(r'[^a-z0-9]+', ' ', string.lower())
    normalized_text.append(tokens)
# 각 문장에서 알파벳소문자와 숫자를 제외하고는 블랭크로 바꿈

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

print(f'총 샘플의 개수: {len(result)}')

총 샘플의 개수: 265335


### 진짜 시작
---

In [11]:
from glove import Corpus, Glove

# 데이터 만들기 -------------------------------------------------
# 훈련 데이터 result로 부터 Glove에서 사용할 동시 등장 행렬을 생성
corpus = Corpus() # = co-occurrence matrix
corpus.fit(result, window=5)    # result를 데이터로 window size는 5로


# 학습 설정 -----------------------------------------------------
glove = Glove(no_components=100     # 아웃풋 벡터(임베딩 결과)의 차원
            , learning_rate=0.05)   # 경사하강법 학습률

glove.fit(corpus.matrix
        , epochs=20                 # 에포크
        , no_threads=4              # 사용할 스레드 개수
        , verbose=True)

# 유사어 검색을 위해서는 sparse matrix 의 각 row, column index 에 해당하는 vocabulary 의 정보가 필요
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [14]:
# 학습 완료 되었으니 유사한 단어 뽑기 해보자
model_result1 = glove.most_similar('woman')
print(model_result1)

[('man', 0.9562144135473687), ('guy', 0.8876158869398776), ('girl', 0.8570906030254912), ('kid', 0.851567422658002)]


In [16]:
model_result2=glove.most_similar("cat")
print(model_result2)

[('battery', 0.8806458422392436), ('physician', 0.8751692764712892), ('sport', 0.8713512761107577), ('dog', 0.8667326054158758)]


In [27]:
model_result3=glove.most_similar("money")
print(model_result3)

[('better', 0.7623932649608524), ('spend', 0.750545642413745), ('get', 0.7306802914490503), ('lot', 0.7305354247041402)]


In [18]:
model_result4=glove.most_similar("water")
print(model_result4)

[('air', 0.8537416799403803), ('fresh', 0.8386465245295099), ('clean', 0.8247012541328808), ('food', 0.8177883595201297)]


In [26]:
model_result5=glove.most_similar("smart")
print(model_result5)

[('quite', 0.7854522161275729), ('rich', 0.7835029402965338), ('healthy', 0.7751506943679994), ('similar', 0.763591558100667)]


In [21]:
model_result6=glove.most_similar("candy")
print(model_result6)

[('symptom', 0.8561671619739152), ('kit', 0.8384330095090121), ('weapon', 0.833907013941906), ('celebration', 0.8270653784203009)]


In [25]:
model_result7=glove.most_similar("scissors")
print(model_result7)

[('encouragement', 0.8971377000148031), ('gloom', 0.8966484737574668), ('stacks', 0.8887300225986902), ('moons', 0.885892264307453)]
